In [1]:
import csv
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
base = "CME"
threshold = 3.5

CME_track = "Input Data/ap2_CME.csv"
Dino_track = "Input Data/ap2_Dino.csv"
CME_detections = "Input Data/detections_CME.csv"
Dino_detections = "Input Data/detections_Dino.csv"

In [43]:
df_CME_track = pd.read_csv(CME_track, header = None)
M_CME_track = df_CME_track.to_numpy()
M_CME_track[:, 2] += M_CME_track[:, 1]-1
M_CME_track[:, 5] = 169 - M_CME_track[:, 5]

df_Dino_track = pd.read_csv(Dino_track)
df_Dino_track = df_Dino_track.drop("max_intensity", axis = 1)
M_Dino_track = df_Dino_track.to_numpy()
M_Dino_track[:, 1:3] += 1

df_CME_detections = pd.read_csv(CME_detections, header = None)
M_CME_detections = df_CME_detections.to_numpy()
#M_CME[:, 4] = M_CME[:, 4]-1

df_Dino_detections = pd.read_csv(Dino_detections)
df_Dino_detections = df_Dino_detections[["timepoint","label_id","x","y","z","intensity","max_intensity"]]
df_Dino_detections = df_Dino_detections.drop("max_intensity", axis=1)
M_Dino_detections = df_Dino_detections.to_numpy()
M_Dino_detections[:, 0] += 1


if base == "CME":
    comp_list = np.zeros((len(M_CME_track),27), dtype=object)
    M_x = M_CME_track
    M_y = M_Dino_track
    sec = "Dino"
if base == "Dino":
    comp_list = np.zeros((len(M_Dino_track),27), dtype=object)
    M_x = M_Dino_track
    M_y = M_CME_track
    sec = "CME"

M_z = M_CME_detections
M_w = M_Dino_detections

In [44]:
t_y = defaultdict(list)
t_z = defaultdict(list)
t_w = defaultdict(list)

for vec_y in M_y:
    t_y[vec_y[2]].append(vec_y)
for t_val in t_y:
    t_y[t_val] = np.array(t_y[t_val])

for vec_z in M_z:
    t_z[vec_z[0]].append(vec_z)
for t_val in t_z:
    t_z[t_val] = np.array(t_z[t_val])

for vec_w in M_w:
    t_w[vec_w[0]].append(vec_w)
for t_val in t_w:
    t_w[t_val] = np.array(t_w[t_val])

for i, vec_x in enumerate(M_x):
    t_val = vec_x[2]
    y_group = t_y[t_val]
    diffs = y_group[:, 3:6] - vec_x[3:6]
    dists = np.linalg.norm(diffs, axis=1)
    below = np.where(dists < threshold)[0]

    z_group = t_z[t_val]
    z_diffs = z_group[:, 2:5] - vec_x[3:6]
    z_dists = np.linalg.norm(z_diffs, axis=1)
    z_below = np.where(z_dists < threshold)[0]

    w_group = t_w[t_val]
    w_diffs = w_group[:, 2:5] - vec_x[3:6]
    w_dists = np.linalg.norm(w_diffs, axis=1)
    w_below = np.where(w_dists < threshold)[0]

    if len(below) > 0:
        y_ids = ";".join(str(int(y)) for y in y_group[below, 0])
        y_TL = ";".join(str(y) for y in y_group[below, 7])
        y_x = ";".join(str(y) for y in y_group[below, 3])
        y_y = ";".join(str(y) for y in y_group[below, 4])
        y_z = ";".join(str(y) for y in y_group[below, 5])
        y_FI = ";".join(str(y) for y in y_group[below, 6])
        dists_str = ";".join(f"{dists[j]:.2f}" for j in below)
    else:
        y_ids = ""
        y_TL = ""
        y_x = ""
        y_y = ""
        y_z = ""
        y_FI = ""
        dists_str = ""

    if len(z_below) > 0:
        z_ids = ";".join(str(z) for z in z_group[z_below, 1])
        z_x = ";".join(str(z) for z in z_group[z_below, 2])
        z_y = ";".join(str(z) for z in z_group[z_below, 3])
        z_z = ";".join(str(z) for z in z_group[z_below, 4])
        z_FI = ";".join(str(z) for z in z_group[z_below, 5])
        z_dists_str = ";".join(f"{z_dists[j]:.2f}" for j in z_below)
    else:
        z_ids = ""
        z_x = ""
        z_y = ""
        z_z = ""
        z_FI = ""
        z_dists_str = ""

    if len(w_below) > 0:
        w_ids = ";".join(str(w) for w in w_group[w_below, 1])
        w_x = ";".join(str(w) for w in w_group[w_below, 2])
        w_y = ";".join(str(w) for w in w_group[w_below, 3])
        w_z = ";".join(str(w) for w in w_group[w_below, 4])
        w_FI = ";".join(str(w) for w in w_group[w_below, 5])
        w_dists_str = ";".join(f"{w_dists[j]:.2f}" for j in w_below)
    else:
        w_ids = ""
        w_x = ""
        w_y = ""
        w_z = ""
        w_FI = ""
        w_dists_str = ""

    comp_list[i, 0:8] = vec_x
    comp_list[i, 8:27] = y_ids, y_TL, y_x, y_y, y_z, y_FI, dists_str, z_ids, z_x, z_y, z_z, z_FI, z_dists_str, w_ids, w_x, w_y, w_z, w_FI, w_dists_str

comp_list = np.delete(comp_list, 1, axis=1)

In [45]:
a = pd.DataFrame(comp_list)
a.to_csv("a.csv", index=False)
a

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1.0,1.0,316.79,151.61,34.96,175.25,99.0,339,74.0,315.22787,...,151.61,34.965,175.25,0.00,339.0,315.22787,151.61403,36.871914,341.0,2.47
1,1.0,2.0,316.91,151.95,35.18,218.14,99.0,339,74.0,315.39215,...,151.95,35.184,218.14,0.00,368.0,315.39215,151.58618,36.012856,352.5,1.77
2,1.0,3.0,316.99,151.34,35.33,218.74,99.0,339,74.0,315.74518,...,151.34,35.329,218.74,0.00,328.0,315.74518,150.99426,36.58273,363.0,1.80
3,1.0,4.0,317.09,152.22,35.24,264.62,99.0,339,74.0,315.99518,...,152.22,35.24,264.62,0.00,303.0,315.99518,151.59639,35.91591,378.0,1.43
4,1.0,5.0,316.93,151.54,35.0,247.41,99.0,339,74.0,316.20465,...,151.54,35.0,247.41,0.00,344.0,316.20465,151.08432,35.901264,361.0,1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77853,6297.0,99.0,516.34,265.71,146.049,31.553,2.0,,,,...,265.71,146.05,31.553,0.00,,,,,,
77854,6298.0,98.0,548.65,294.81,151.196,61.88,2.0,,,,...,294.81,151.2,61.88,0.00,,,,,,
77855,6298.0,99.0,548.09,294.26,149.239,59.079,2.0,,,,...,294.26,149.24,59.079,0.00,,,,,,
77856,6299.0,98.0,509.9,300.23,153.892,64.359,2.0,,,,...,300.23,153.89,64.359,0.00,,,,,,


In [48]:
id_col = 0
cols1 = [2, 9, 15, 21]
cols2 = [3, 10, 16, 22]
cols3 = [4, 11, 17, 23]

ids = np.unique(comp_list[:, id_col])
results = []

for id_val in ids:
    group = comp_list[comp_list[:, id_col] == id_val]

    vals1 = []
    for col in cols1:
        for cell in group[:, col]:
            vals1.extend([float(x) for x in str(cell).split(';') if x != ''])

    vals2 = []
    for col in cols2:
        for cell in group[:, col]:
            vals2.extend([float(x) for x in str(cell).split(';') if x != ''])

    vals3 = []
    for col in cols3:
        for cell in group[:, col]:
            vals3.extend([float(x) for x in str(cell).split(';') if x != ''])
    
    results.append({
        "ID (CME)": id_val,
        "xmin": min(vals1) if vals1 else np.nan,
        "xmax": max(vals1) if vals1 else np.nan,
        "ymin": min(vals2) if vals2 else np.nan,
        "ymax": max(vals2) if vals2 else np.nan,
        "zmin": min(vals3) if vals3 else np.nan,
        "zmax": max(vals3) if vals3 else np.nan,
    })

result_df = pd.DataFrame(results)

      ID (CME)       xmin       xmax       ymin       ymax        zmin  \
0          1.0  312.41000  319.60000  147.91000  156.89000   31.227000   
1          2.0  315.24268  322.53000  327.01500  330.90000   35.922077   
2          3.0  321.82850  328.29000  158.38727  166.15271   38.196000   
3          4.0  321.48462  326.32000  558.75670  567.41710   44.939380   
4          5.0  332.94772  339.01000  435.83786  445.29000   50.369923   
...        ...        ...        ...        ...        ...         ...   
6204    6295.0  601.99000  604.18585  195.54000  196.74287  143.492130   
6205    6296.0  532.57495  535.16000  348.99948  352.02000  140.908000   
6206    6297.0  516.34000  519.46790  265.71000  267.11000  145.839860   
6207    6298.0  548.09000  548.65000  294.26000  294.81000  149.239000   
6208    6299.0  508.82000  509.90000  299.54000  300.23000  153.890000   

            zmax  
0      36.871914  
1      40.303135  
2      42.410130  
3      50.000000  
4      54.263000

In [58]:
result_df.to_csv("CME Base Output/IDCoordinates.csv")